In [1]:
# mlrun: start-code

In [2]:
from cloudpickle import load
import numpy as np
from typing import List
import mlrun

class ClassifierModel(mlrun.serving.V2ModelServer):
    def load(self):
        """load and initialize the model and/or other elements"""
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample."""
        feats = np.asarray(body['inputs'])
        result: np.ndarray = self.model.predict(feats)
        return result.tolist()

In [3]:
# mlrun: end-code

In [4]:
from mlrun import code_to_function, auto_mount

serving_fn = code_to_function(
    name='model-server',
    project="royal-cyber",
    kind='serving',
    image='mlrun/mlrun'
).apply(auto_mount())

serving_fn.spec.default_class = 'ClassifierModel'

serving_fn.add_model('lr', model_path="store://models/royal-cyber/my-training-job-tracking-main_LogisticRegression_model#0:latest@32c8cd1816a14b5eb49d99ab92521b08")
serving_fn.add_model('rf', model_path="store://models/royal-cyber/my-training-job-tracking-main_RandomForestClassifier_model#0:latest@32c8cd1816a14b5eb49d99ab92521b08")

In [5]:
function_address = serving_fn.deploy()

> 2021-08-20 22:50:08,915 [info] Starting remote function deploy
2021-08-20 22:50:09  (info) Deploying function
2021-08-20 22:50:09  (info) Building
2021-08-20 22:50:09  (info) Staging files and preparing base images
2021-08-20 22:50:09  (info) Building processor image
2021-08-20 22:50:13  (info) Build complete
2021-08-20 22:50:21  (info) Function deploy complete
> 2021-08-20 22:50:21,646 [info] function deployed, address=default-tenant.app.us-sales-eks.iguazio-cd0.com:30624


In [12]:
import json
import requests

X_test = [[5.1, 3.5, 1.4, 0.2],[7.7, 3.8, 6.7, 2.2]]
data = json.dumps({"inputs" : X_test})
data

'{"inputs": [[5.1, 3.5, 1.4, 0.2], [7.7, 3.8, 6.7, 2.2]]}'

In [27]:
for model in ["lr", "rf"]:
    # Inference via serving function
    print(serving_fn.invoke(f'/v2/models/{model}/infer', data))
    
    # Inference via HTTP post request
    print(requests.post(url=f"{function_address}/v2/models/{model}/infer", data=data).json())

{'id': 'ce315b86-fdd7-4384-9697-c4f9d74459e9', 'model_name': 'lr', 'outputs': [0, 2]}
{'id': '8c782ec6-20f7-486d-acb6-104c93e3d323', 'model_name': 'lr', 'outputs': [0, 2]}
{'id': '053558c5-ca47-4576-85b6-572b10fe3644', 'model_name': 'rf', 'outputs': [0, 2]}
{'id': '83d2ce43-18c7-4bfd-a56d-3f64f8f87e4a', 'model_name': 'rf', 'outputs': [0, 2]}
